In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from itertools import product
pd.set_option('display.max_columns', None)

In [34]:
sales_train = pd.read_csv(r'G:\machine learning\kaggle\predict future sales\\sales_train.csv')
item_categories = pd.read_csv(r'G:\machine learning\kaggle\predict future sales\\item_categories-translated.csv')
items = pd.read_csv(r'G:\machine learning\kaggle\predict future sales\\items.csv')
test = pd.read_csv(r'G:\machine learning\kaggle\predict future sales\\test.csv')
shops = pd.read_csv(r'G:\machine learning\kaggle\predict future sales\\shops-translated.csv')

In [ ]:
sales_train['date_block_num'] = sales_train['date_block_num'].astype('int8')
sales_train['shop_id'] = sales_train['shop_id'].astype('int16')
sales_train['item_id'] = sales_train['item_id'].astype('int16')
sales_train['item_price'] = sales_train['item_price'].astype('float16')
sales_train['item_cnt_day'] = sales_train['item_cnt_day'].astype('float16')
item_categories['item_category_id'] = item_categories['item_category_id'].astype('int16')
items['item_id'] = items['item_id'].astype('int16')
items['item_category_id'] = items['item_category_id'].astype('int16')
test['shop_id'] = test['shop_id'].astype('int16')
test['item_id'] = test['item_id'].astype('int16')
test['date_block_num'] = 34
test['date_block_num'] = test['date_block_num'].astype('int8')

In [ ]:
sub = pd.DataFrame()
sub['ID'] = test['ID']
test = test.drop(['ID'],axis=1)

In [ ]:
sales_train = sales_train.drop_duplicates()
sales_train = sales_train[sales_train['item_price']<=45000]
sales_train = sales_train[sales_train['item_cnt_day']<=1000]
sales_train.loc[sales_train['shop_id']==0, 'shop_id']=57
test.loc[test['shop_id']==0, 'shop_id']=57
sales_train.loc[sales_train['shop_id']==1, 'shop_id']=58
test.loc[test['shop_id']==1, 'shop_id']=58
sales_train.loc[sales_train['shop_id']==10, 'shop_id']=11
test.loc[test['shop_id']==10, 'shop_id']=11
median = sales_train[(sales_train['shop_id']==32)&(sales_train['item_id']==2973)&(sales_train['date_block_num']==4)&(sales_train['item_price']>0)].item_price.median()
sales_train.loc[sales_train['item_price']<=0, 'item_price'] = median

In [ ]:
shops['city'] = shops['shop_name_translated'].apply(lambda x:x.split(' ')[0])
shops['city_code'] = LabelEncoder().fit_transform(shops['city'])
shops = shops[['shop_id', 'city_code']]

In [ ]:
item_categories['item_name'] = item_categories['item_category_name_translated'].apply(lambda x:x.split(' - ')[0].strip())
item_categories['item_name_code'] = LabelEncoder().fit_transform(item_categories['item_name'])
item_categories['item_subname'] = item_categories['item_category_name_translated'].apply(lambda x:x.split(' - ')[-1].strip())
item_categories['item_subname_code'] = LabelEncoder().fit_transform(item_categories['item_subname'])
item_categories = item_categories.drop(['item_category_name_translated','item_name','item_subname'],axis=1)

In [ ]:
cols = ['date_block_num', 'shop_id', 'item_id']
combine_items = []
for i in range(34):
    sales = sales_train[sales_train['date_block_num']==i]
    combine_items.append(np.array(list(product([i], sales['shop_id'].unique(), sales['item_id'].unique())),dtype='int16'))
combine_items = pd.DataFrame(np.vstack(combine_items),columns=cols)
combine_items['date_block_num'] = combine_items['date_block_num'].astype('int8')
combine_items['shop_id'] = combine_items['shop_id'].astype('int16')
combine_items['item_id'] = combine_items['item_id'].astype('int16')
combine_items.sort_values(cols,inplace=True)
combine_items = combine_items.reset_index(drop=True)

In [ ]:
sales_train['revenue'] = sales_train['item_price']*sales_train['item_cnt_day']

In [ ]:
group = sales_train.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day': ['sum']})
group.columns = ['item_cnt_month']
group.reset_index(inplace=True)

combine_items = pd.merge(combine_items, group, on=['date_block_num','shop_id','item_id'], how='left')
combine_items['item_cnt_month'] = combine_items['item_cnt_month'].fillna(0).clip(0,20).astype(np.float16)

In [ ]:
test_size = len(test)
all_data = pd.concat([combine_items, test],ignore_index=True, sort=False,keys=cols)
all_data.fillna(0, inplace=True)

In [ ]:
items_cats = pd.merge(items, item_categories, on=['item_category_id'], how='left')
items_cats = items_cats.drop(['item_name'], axis=1)
all_data = pd.merge(all_data, shops, on=['shop_id'], how='left')
all_data = pd.merge(all_data, items_cats, on=['item_id'], how='left')

all_data['city_code'] = all_data['city_code'].astype('int16')
all_data['item_name_code'] = all_data['item_name_code'].astype('int16')
all_data['item_subname_code'] = all_data['item_subname_code'].astype('int16')

In [ ]:
def lag_feature(df, lags, cols):
    temp = df[['date_block_num', 'shop_id', 'item_id', cols]]
    for i in lags:
        shifted = temp.copy()
        shifted.columns = ['date_block_num', 'shop_id', 'item_id', cols+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num', 'shop_id', 'item_id'], how='left')
    return df

all_data = lag_feature(all_data, [1,2,3,6,12], 'item_cnt_month')

In [ ]:
group = all_data.groupby(['date_block_num', 'shop_id']).agg({'item_cnt_month':['mean']})
group.columns = ['shop_month_avg_cnt']
group.reset_index(inplace=True)

all_data = pd.merge(all_data, group, on=['date_block_num', 'shop_id'], how='left')
all_data = lag_feature(all_data,[1,2,3,6,12], 'shop_month_avg_cnt')
all_data = all_data.drop(['shop_month_avg_cnt'],axis=1)

In [ ]:
group = all_data.groupby(['date_block_num', 'item_id']).agg({'item_cnt_month':['mean']})
group.columns = ['item_month_avg_cnt']
group.reset_index(inplace=True)

all_data = pd.merge(all_data, group,on=['date_block_num', 'item_id'], how='left')
all_data = lag_feature(all_data,[1,2,3,6,12], 'item_month_avg_cnt')
all_data = all_data.drop(['item_month_avg_cnt'],axis=1)

In [ ]:
group = all_data.groupby(['date_block_num']).agg({'item_cnt_month':['mean']})
group.columns = ['month_avg_cnt']
group.reset_index(inplace=True)

all_data = pd.merge(all_data, group,on=['date_block_num'], how='left')
all_data = lag_feature(all_data,[1,2], 'month_avg_cnt')
all_data = all_data.drop(['month_avg_cnt'],axis=1)

In [ ]:
group = all_data.groupby(['date_block_num','city_code']).agg({'item_cnt_month':['mean']})
group.columns = ['city_avg_cnt']
group.reset_index(inplace=True)

all_data = pd.merge(all_data, group,on=['date_block_num', 'city_code'], how='left')
all_data = lag_feature(all_data,[1,2], 'city_avg_cnt')
all_data = all_data.drop(['city_avg_cnt'],axis=1)

In [ ]:
group = all_data.groupby(['date_block_num','item_name_code']).agg({'item_cnt_month':['mean']})
group.columns = ['item_name_avg_cnt']
group.reset_index(inplace=True)

all_data = pd.merge(all_data, group,on=['date_block_num', 'item_name_code'], how='left')
all_data = lag_feature(all_data,[1,2], 'item_name_avg_cnt')
all_data = all_data.drop(['item_name_avg_cnt'],axis=1)

In [ ]:
group = all_data.groupby(['date_block_num','item_subname_code']).agg({'item_cnt_month':['mean']})
group.columns = ['item_subname_avg_cnt']
group.reset_index(inplace=True)

all_data = pd.merge(all_data, group,on=['date_block_num', 'item_subname_code'], how='left')
all_data = lag_feature(all_data,[1,2], 'item_subname_avg_cnt')
all_data = all_data.drop(['item_subname_avg_cnt'],axis=1)

In [ ]:
group = all_data.groupby(['date_block_num','shop_id','item_name_code']).agg({'item_cnt_month':['mean']})
group.columns = ['shop_item_name_avg_cnt']
group.reset_index(inplace=True)

all_data = pd.merge(all_data, group,on=['date_block_num', 'shop_id', 'item_name_code'], how='left')
all_data = lag_feature(all_data,[1,2], 'shop_item_name_avg_cnt')
all_data = all_data.drop(['shop_item_name_avg_cnt'],axis=1)

In [ ]:
group = all_data.groupby(['date_block_num','shop_id','item_subname_code']).agg({'item_cnt_month':['mean']})
group.columns = ['shop_item_subname_avg_cnt']
group.reset_index(inplace=True)

all_data = pd.merge(all_data, group,on=['date_block_num', 'shop_id','item_subname_code'], how='left')
all_data = lag_feature(all_data,[1,2], 'shop_item_subname_avg_cnt')
all_data = all_data.drop(['shop_item_subname_avg_cnt'],axis=1)

In [ ]:
group = all_data.groupby(['date_block_num','shop_id','city_code']).agg({'item_cnt_month':['mean']})
group.columns = ['shop_city_avg_cnt']
group.reset_index(inplace=True)

all_data = pd.merge(all_data, group,on=['date_block_num', 'shop_id','city_code'], how='left')
all_data = lag_feature(all_data,[1,2], 'shop_city_avg_cnt')
all_data = all_data.drop(['shop_city_avg_cnt'],axis=1)

In [ ]:
group = all_data.groupby(['date_block_num','item_id','city_code']).agg({'item_cnt_month':['mean']})
group.columns = ['item_city_avg_cnt']
group.reset_index(inplace=True)

all_data = pd.merge(all_data, group,on=['date_block_num', 'item_id','city_code'], how='left')
all_data = lag_feature(all_data,[1,2], 'item_city_avg_cnt')
all_data = all_data.drop(['item_city_avg_cnt'],axis=1)

In [ ]:
group = sales_train.groupby(['date_block_num','item_id']).agg({'item_price':['mean']})
group.columns = ['date_item_avg_price']
group.reset_index(inplace=True)

all_data = pd.merge(all_data, group, on=['date_block_num', 'item_id'], how='left')
all_data = lag_feature(all_data, [1,2,3,4,5,6], 'date_item_avg_price')
all_data = all_data.drop(['date_item_avg_price'],axis=1)

In [ ]:
group = sales_train.groupby(['item_id']).agg({'item_price':['mean']})
group.columns=['item_avg_price']
group.reset_index(inplace=True)
all_data = pd.merge(all_data, group, on=['item_id'], how='left')

In [ ]:
for i in range(1,7):
    all_data['delta_price_lag_'+str(i)] = (all_data['date_item_avg_price_lag_'+str(i)] - all_data['item_avg_price'])/all_data['item_avg_price']
def select_trend(row):
    for i in [1,2,3,4,5,6]:
        if row['delta_price_lag_'+str(i)]:
            return row['delta_price_lag_'+str(i)]
    return 0
    
all_data['delta_price_lag'] = all_data.apply(select_trend, axis=1)
all_data['delta_price_lag'] = all_data['delta_price_lag'].astype(np.float16)
all_data['delta_price_lag'].fillna(0, inplace=True)
# all_data['delta_price_lag'] = all_data[['delta_price_lag_1', 'delta_price_lag_2', 'delta_price_lag_3', 'delta_price_lag_4', 'delta_price_lag_5', 'delta_price_lag_6']].bfill(axis=1).iloc[:,0]
drop_features = ['item_avg_price']
for i in range(1,7):
    drop_features.append('date_item_avg_price_lag_'+str(i))
    drop_features.append('delta_price_lag_'+str(i))
all_data = all_data.drop(columns=drop_features)

In [ ]:
group = sales_train.groupby(['date_block_num','shop_id']).agg({'revenue':['sum']})
group.columns = ['date_shop_revenue']
group.reset_index(inplace=True)
all_data = pd.merge(all_data, group, on=['date_block_num', 'shop_id'], how='left')

group = all_data.groupby(['shop_id']).agg({'date_shop_revenue':['mean']})
group.columns = ['shop_avg_revenue']
group.reset_index(inplace=True)
all_data = pd.merge(all_data, group, on=['shop_id'], how='left')

all_data['delta_revenue'] = (all_data['date_shop_revenue'] - all_data['shop_avg_revenue'])/all_data['shop_avg_revenue']
all_data = lag_feature(all_data, [1], 'delta_revenue')
all_data = all_data.drop(['date_shop_revenue', 'shop_avg_revenue', 'delta_revenue'],axis=1)

In [ ]:
all_data['month'] = (all_data['date_block_num']+1)%12
days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
all_data['day'] = all_data['month'].map(days).astype('int8')

In [ ]:
temp = all_data[all_data['date_block_num']>11]
temp = temp.drop(['delta_revenue_lag_1'], axis=1)
temp = temp.fillna(0)

In [ ]:
X = temp[temp['date_block_num']<34]
Y = X['item_cnt_month']
X = X.drop(['item_cnt_month'], axis=1)
test = temp[temp['date_block_num']==34]
test = test.drop(['item_cnt_month'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
from lightgbm.sklearn import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
kfold = KFold(n_splits=15, shuffle = True, random_state= 12)
lgb_model = LGBMRegressor(n_jobs=-1,
                          learning_rate=0.05,
                          n_estimators=1000,
                          max_depth=6,
                          num_leaves=63,
                          reg_alpha=1, 
                          reg_lambda=1,
                          min_child_samples=128,
                          min_split_gain=0.5,
                          colsample_bytree=0.6)
for train, test in kfold.split(X):
    X_train = X.iloc[train]
    y_train = Y.iloc[train]
    X_test = X.iloc[test]
    y_test = Y.iloc[test]
    lgb_model.fit(X_train,y_train)
    y_pred = lgb_model.predict(X=X_test)
    e = mean_squared_error(y_true=y_test,y_pred=y_pred)
    print(e)

In [ ]:
lgb_pred = lgb_model.predict(test)
sub['item_cnt_month'] = lgb_pred
sub.to_csv('lgb_submission.csv', index=False)